In [8]:
import pandas as pd
import sqlite3 as sql
import pyodbc

Northwind = {
    'servername': r'DAVIDS_PC',
    'database': 'Northwind'
}

Adventure = {
    'servername': r'DAVIDS_PC',
    'database': 'AdventureWorks'
}


In [9]:
aenc_connection = sql.connect("../data/aenc.sqlite")
adventure_connection = pyodbc.connect('DRIVER={SQL Server};SERVER=' + Adventure['servername'] + ';DATABASE=' + Adventure['database'] + ';Trusted_Connection=yes')
northwind_connection = pyodbc.connect('DRIVER={SQL Server};SERVER=' + Northwind['servername'] + ';DATABASE=' + Northwind['database'] + ';Trusted_Connection=yes')

In [10]:
product1 = pd.read_sql_query("select * from product", aenc_connection)
product2 = pd.read_sql_query(f"select * from Production.Product", adventure_connection)
product3 = pd.read_sql_query("select * from products", northwind_connection)

products = pd.DataFrame()

C:\Users\david\AppData\Local\Temp\ipykernel_14692\988504725.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product2 = pd.read_sql_query(f"select * from Production.Product", adventure_connection)
C:\Users\david\AppData\Local\Temp\ipykernel_14692\988504725.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product3 = pd.read_sql_query("select * from products", northwind_connection)


In [11]:
def add_products(products, id, database, name, category_name, product_description, product_size, quantity_per_unit, unit_price, discontinued, inventory_quantity):
    return pd.concat([products, pd.DataFrame({
        'Id': [id],
        'Database': [database],
        'Name': [name],
        'CategoryName': [category_name],
        'ProductDescription': [product_description],
        'ProductSize': [product_size],
        'QuantityPerUnit': [quantity_per_unit],
        'UnitPrice': [unit_price],
        'Discontinued': [discontinued],
        'InventoryQuantity': [inventory_quantity],
    })])


In [12]:
for index, row in product1.loc[:, ['id', 'name', 'Category', 'prod_size', 'quantity', 'unit_price']].iterrows():
    products = add_products(products, 
        row['id'],
        'AE',
        row['name'],
        row['Category'],
        '',
        row['prod_size'],
        1,
        row['unit_price'],
        False,
        row['quantity']
    )

In [17]:
for index, row in product2.loc[:, ['ProductID', 'Name', 'ProductLine', 'Size', 'Color', 'Weight', 'DiscontinuedDate']].iterrows():
    result = pd.read_sql_query(f'SELECT top 1 Quantity from Production.ProductInventory where ProductID like {row['ProductID']} ORDER BY ModifiedDate DESC', adventure_connection)

    products = add_products(products, 
        row['ProductID'],
        'AW',
        row['Name'],
        row['ProductLine'],
        f"Color: {row['Color']} Weight: {row['Weight']}",
        row['Size'],
        1,
        None,
        False,
        result['Quantity'].iloc[0] if not result.empty else None
    )

C:\Users\david\AppData\Local\Temp\ipykernel_14692\123603694.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(f'SELECT top 1 Quantity from Production.ProductInventory where ProductID like {row['ProductID']} ORDER BY ModifiedDate DESC', adventure_connection)
C:\Users\david\AppData\Local\Temp\ipykernel_14692\123603694.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(f'SELECT top 1 Quantity from Production.ProductInventory where ProductID like {row['ProductID']} ORDER BY ModifiedDate DESC', adventure_connection)
C:\Users\david\AppData\Local\Temp\ipykernel_14692\123603694.py:2: UserWarning: pandas only supports SQLAlchemy c

In [18]:
for index, row in product3.loc[:, ['ProductID', 'ProductName', 'CategoryID', 'QuantityPerUnit', 'UnitPrice', 'UnitsInStock', 'Discontinued']].iterrows():
    result = pd.read_sql_query(f'select CategoryName from Categories where CategoryID = {row['CategoryID']}', northwind_connection)

    products = add_products(products, 
        row['ProductID'],
        'NW',
        row['ProductName'],
        result['CategoryName'].iloc[0] if not result.empty else None,
        '',
        '',
        row['QuantityPerUnit'],
        row['UnitPrice'],
        row['Discontinued'],
        row['UnitsInStock']
    )

C:\Users\david\AppData\Local\Temp\ipykernel_14692\2392741610.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(f'select CategoryName from Categories where CategoryID = {row['CategoryID']}', northwind_connection)


In [22]:
products.loc[products['Database'] == 'AE', :]

,Id,Database,Name,CategoryName,ProductDescription,ProductSize,QuantityPerUnit,UnitPrice,Discontinued,InventoryQuantity
0,300,AE,Tee Shirt,Clothes,,Small,1,9,False,18
0,301,AE,Tee Shirt,Clothes,,Medium,1,14,False,39
0,302,AE,Tee Shirt,Clothes,,One size fits all,1,14,False,72
0,400,AE,Baseball Cap,Accessories,,One size fits all,1,9,False,92
0,401,AE,Baseball Cap,Accessories,,One size fits all,1,10,False,12
0,500,AE,Visor,Accessories,,One size fits all,1,7,False,36
0,501,AE,Visor,Accessories,,One size fits all,1,7,False,28
0,600,AE,Sweatshirt,Clothes,,Large,1,24,False,39
0,601,AE,Sweatshirt,Clothes,,Large,1,24,False,32
0,700,AE,Shorts,Clothes,,Medium,1,15,False,80
